# Diffusion Process Animation with Manim

This notebook demonstrates the core concepts of Diffusion Models: the Forward Process (adding noise) and the Reverse Process (denoising).

## 1. Setup

We need to import Manim and configure it for the notebook environment.

> **Note:** Manim requires `ffmpeg` to render videos. If `ffmpeg` is not installed on your system, you may see errors when running these cells.

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

In [ ]:
from manim import *
import numpy as np
config.media_embed = True

## 2. Forward Diffusion: Adding Noise

In the forward process, we start with a clean data point and iteratively add Gaussian noise until it becomes pure noise.

Mathematically, this is expressed as:
$q(x_t | x_{t-1}) = \mathcal{N}(x_t; \sqrt{1-\beta_t}x_{t-1}, \beta_t \mathbf{I})$

In [ ]:
%%manim -v WARNING -qm ForwardDiffusion

class ForwardDiffusion(Scene):
    def construct(self):
        # Clean state: A grid of dots forming a shape
        dots = VGroup(*[Dot(point=[x, y, 0], radius=0.04, color=BLUE) 
                       for x in np.arange(-2, 2.1, 0.4) 
                       for y in np.arange(-2, 2.1, 0.4)])
        
        title = Text("Forward Diffusion: Adding Noise").to_edge(UP)
        t_label = Variable(0, Text("Step t"), var_type=Integer).to_edge(RIGHT)
        
        self.add(title, dots, t_label)
        self.wait(1)
        
        # Store original positions
        original_positions = [dot.get_center() for dot in dots]
        
        steps = 5
        for t in range(1, steps + 1):
            self.play(t_label.tracker.animate.set_value(t), run_time=0.5)
            
            animations = []
            noise_scale = t * 0.4
            for i, dot in enumerate(dots):
                # Add random displacement
                noise = np.random.normal(0, noise_scale, 3)
                noise[2] = 0 
                animations.append(dot.animate.move_to(original_positions[i] + noise))
            
            self.play(*animations, run_time=1)
            self.wait(0.5)
        
        conclusion = Text("Result: Pure Gaussian Noise").scale(0.8).next_to(dots, DOWN)
        self.play(Write(conclusion))
        self.wait(2)

## 3. Reverse Diffusion: Denoising

The reverse process is what the model learns. It starts from noise and attempts to remove it step-by-step to recover the data.

The model $p_\theta(x_{t-1} | x_t)$ learns to predict the noise added at each step.

In [ ]:
%%manim -v WARNING -qm ReverseDiffusion

class ReverseDiffusion(Scene):
    def construct(self):
        # Start with noisy state
        num_dots = 121
        dots = VGroup(*[Dot(point=[np.random.normal(0, 2), np.random.normal(0, 2), 0], 
                           radius=0.05, color=RED) for _ in range(num_dots)])
        
        # Target positions (the grid)
        target_positions = [np.array([x, y, 0]) 
                           for x in np.arange(-2, 2.1, 0.4) 
                           for y in np.arange(-2, 2.1, 0.4)]
        
        title = Text("Reverse Diffusion: Learning to Denoise").to_edge(UP)
        model_box = Rectangle(height=1.5, width=3, color=WHITE).to_edge(LEFT, buff=0.5)
        model_text = Text("Neural\nNetwork", color=YELLOW).scale(0.5).move_to(model_box)
        
        self.add(title, dots, model_box, model_text)
        self.wait(1)
        
        steps = 5
        for t in range(steps, -1, -1):
            # Show the model predicting noise
            self.play(model_box.animate.set_color(YELLOW), run_time=0.2)
            
            animations = []
            for i, dot in enumerate(dots):
                alpha = (steps - t) / steps
                target = target_positions[i]
                current = dot.get_center()
                # Iteratively move closer to target
                next_pos = current + (target - current) * 0.4
                animations.append(dot.animate.move_to(next_pos).set_color(interpolate_color(RED, BLUE, alpha)))
            
            self.play(*animations, model_box.animate.set_color(WHITE), run_time=1)
            
        self.wait(1)
        success = Text("Data Recovered!", color=GREEN).next_to(dots, DOWN)
        self.play(Write(success))
        self.wait(2)

## 4. Key Concepts

- **Noise Schedule (Beta):** Controls how much noise is added at each step.
- **Forward Process:** Fixed and non-learnable. It gradually destroys data.
- **Reverse Process:** Learned by a neural network. It recovers data from noise.
- **Objective:** The network is trained to minimize the difference between the actual noise added and the noise it predicts.